# NY Fed Data

- Empire M-PMI | .csv file | [NY Fed](https://www.newyorkfed.org/survey/empire/empiresurvey_overview#tabs-1)
- Survey of Consumer Expectations | .csv file | [NY Fed](https://www.newyorkfed.org/microeconomics/sce#/)
- Consumer Debt and Credit | .csv file | [NY Fed](https://www.newyorkfed.org/microeconomics/hhdc)
- Underlying Inflation Gauge | .csv file | [NY Fed](https://www.newyorkfed.org/research/policy/underlying-inflation-gauge)
- Weekly Economic Index | .csv file | [NY Fed](https://www.newyorkfed.org/research/policy/weekly-economic-index#/interactive)
- Recession Probability | .csv file | [NY Fed](https://www.newyorkfed.org/research/capital_markets/ycfaq#/interactive)
- Supply Chain Index | .csv file | [NY Fed](https://www.newyorkfed.org/research/policy/gscpi#/interactive)


In [1]:
import os
import pandas as pd

# path for the folder "project"
path = "C:\\Users\\pedro\\OneDrive\\NYU\\CSS\\II. Data Skills\\project"
os.chdir(path)

## Empire M-PMI (Empire)

Importing the csv file from the link:

In [15]:
# Seasonally adjusted data:
url = "https://www.newyorkfed.org/medialibrary/media/survey/empire/data/esms_seasonallyadjusted_allseries.csv?la=en"
ny_pmi = pd.read_csv(url)

# non adjusted data:
url_nsa = "https://www.newyorkfed.org/medialibrary/media/survey/empire/data/esms_notseasonallyadjusted_allseries.csv?la=en"
ny_nsa = pd.read_csv(url_nsa)

ny_pmi.head()

,surveyDate,GACDSA,GACISA,GACNSA,GACDISA,NOCDSA,NOCISA,NOCNSA,NOCDISA,SHCDSA,...,AWFNSA,AWFDISA,CEFDSA,CEFISA,CEFNSA,CEFDISA,TSFDSA,TSFISA,TSFNSA,TSFDISA
0,2001-07-31,35.5,22.2,42.3,-13.3,29.4,36.5,34.1,7.0,34.8,...,65.5,15.8,23.9,28.2,47.9,4.2,12.1,30.0,57.9,17.9
1,2001-08-31,30.0,22.0,48.0,-8.0,28.9,28.1,43.0,-0.8,33.4,...,58.9,10.6,24.4,28.1,47.5,3.8,16.6,25.5,58.0,8.9
2,2001-09-30,31.9,19.4,48.7,-12.5,20.0,44.9,35.1,25.0,36.1,...,62.5,16.3,22.4,32.8,44.8,10.3,19.5,22.1,58.4,2.7
3,2001-10-31,37.1,22.4,40.5,-14.7,42.6,24.4,33.0,-18.2,36.4,...,58.5,8.9,17.5,32.1,50.4,14.6,13.9,26.3,59.9,12.4
4,2001-11-30,38.0,16.4,45.6,-21.5,31.6,24.6,43.7,-7.0,26.9,...,61.5,6.0,20.8,32.0,47.2,11.2,15.9,24.6,59.5,8.7


In [16]:
ny_pmi = (
    ny_pmi
    .rename(columns = {"surveyDate":"date"})
    .melt(id_vars="date",var_name = "series_id")
    [["series_id","date","value"]]
)

ny_nsa = (
    ny_nsa
    .rename(columns = {"surveyDate":"date"})
    .melt(id_vars="date",var_name = "series_id")
    [["series_id","date","value"]]
)

# merge:
ny_pmi = pd.concat([ny_pmi,ny_nsa]).reset_index(drop=True)

ny_pmi.head()

,series_id,date,value
0,GACDSA,2001-07-31,35.5
1,GACDSA,2001-08-31,30.0
2,GACDSA,2001-09-30,31.9
3,GACDSA,2001-10-31,37.1
4,GACDSA,2001-11-30,38.0


In [17]:
# adjusting dates.
ny_pmi["date"] = pd.to_datetime(ny_pmi["date"])

ny_pmi["date"] = ny_pmi["date"] + pd.Timedelta(days=1)
ny_pmi["date"] = ny_pmi["date"] - pd.DateOffset(months=1)

# series_id in lower case.
ny_pmi["series_id"] = ny_pmi["series_id"].str.lower()

ny_pmi.head()

,series_id,date,value
0,gacdsa,2001-07-01,35.5
1,gacdsa,2001-08-01,30.0
2,gacdsa,2001-09-01,31.9
3,gacdsa,2001-10-01,37.1
4,gacdsa,2001-11-01,38.0


Building dictionary. The series_id are almost the same from the Philly Fed PMI (see `philly_fed.ipynb`)

In [19]:
# Building dict:
ny_pmi_dict = [["gacdsa","General Activity","Current","Decrease","SA"],
               ["gacdna","General Activity","Current","Decrease","NSA"],
               ["gafdsa","General Activity","Future","Decrease","SA"],
               ["gafdna","General Activity","Future","Decrease","NSA"],
               ["gacnsa","General Activity","Current","No Change","SA"],
               ["gacnna","General Activity","Current","No Change","NSA"],
               ["gafnsa","General Activity","Future","No Change","SA"],
               ["gafnna","General Activity","Future","No Change","NSA"],
               ["gacisa","General Activity","Current","Increase","SA"],
               ["gacina","General Activity","Current","Increase","NSA"],
               ["gafisa","General Activity","Future","Increase","SA"],
               ["gafina","General Activity","Future","Increase","NSA"],
               ["gacdisa","General Activity","Current","Diffusion Index","SA"],
               ["gacdina","General Activity","Current","Diffusion Index","NSA"],
               ["gafdisa","General Activity","Future","Diffusion Index","SA"],
               ["gafdina","General Activity","Future","Diffusion Index","NSA"],
               ["nocdsa","New Orders","Current","Decrease","SA"],
               ["nocdna","New Orders","Current","Decrease","NSA"],
               ["nofdsa","New Orders","Future","Decrease","SA"],
               ["nofdna","New Orders","Future","Decrease","NSA"],
               ["nocnsa","New Orders","Current","No Change","SA"],
               ["nocnna","New Orders","Current","No Change","NSA"],
               ["nofnsa","New Orders","Future","No Change","SA"],
               ["nofnna","New Orders","Future","No Change","NSA"],
               ["nocisa","New Orders","Current","Increase","SA"],
               ["nocina","New Orders","Current","Increase","NSA"],
               ["nofisa","New Orders","Future","Increase","SA"],
               ["nofina","New Orders","Future","Increase","NSA"],
               ["nocdisa","New Orders","Current","Diffusion Index","SA"],
               ["nocdina","New Orders","Current","Diffusion Index","NSA"],
               ["nofdisa","New Orders","Future","Diffusion Index","SA"],
               ["nofdina","New Orders","Future","Diffusion Index","NSA"],
               ["shcdsa","Shipments","Current","Decrease","SA"],
               ["shcdna","Shipments","Current","Decrease","NSA"],
               ["shfdsa","Shipments","Future","Decrease","SA"],
               ["shfdna","Shipments","Future","Decrease","NSA"],
               ["shcnsa","Shipments","Current","No Change","SA"],
               ["shcnna","Shipments","Current","No Change","NSA"],
               ["shfnsa","Shipments","Future","No Change","SA"],
               ["shfnna","Shipments","Future","No Change","NSA"],
               ["shcisa","Shipments","Current","Increase","SA"],
               ["shcina","Shipments","Current","Increase","NSA"],
               ["shfisa","Shipments","Future","Increase","SA"],
               ["shfina","Shipments","Future","Increase","NSA"],
               ["shcdisa","Shipments","Current","Diffusion Index","SA"],
               ["shcdina","Shipments","Current","Diffusion Index","NSA"],
               ["shfdisa","Shipments","Future","Diffusion Index","SA"],
               ["shfdina","Shipments","Future","Diffusion Index","NSA"],
               ["dtcdsa","Delivery Time","Current","Decrease","SA"],
               ["dtcdna","Delivery Time","Current","Decrease","NSA"],
               ["dtfdsa","Delivery Time","Future","Decrease","SA"],
               ["dtfdna","Delivery Time","Future","Decrease","NSA"],
               ["dtcnsa","Delivery Time","Current","No Change","SA"],
               ["dtcnna","Delivery Time","Current","No Change","NSA"],
               ["dtfnsa","Delivery Time","Future","No Change","SA"],
               ["dtfnna","Delivery Time","Future","No Change","NSA"],
               ["dtcisa","Delivery Time","Current","Increase","SA"],
               ["dtcina","Delivery Time","Current","Increase","NSA"],
               ["dtfisa","Delivery Time","Future","Increase","SA"],
               ["dtfina","Delivery Time","Future","Increase","NSA"],
               ["dtcdisa","Delivery Time","Current","Diffusion Index","SA"],
               ["dtcdina","Delivery Time","Current","Diffusion Index","NSA"],
               ["dtfdisa","Delivery Time","Future","Diffusion Index","SA"],
               ["dtfdina","Delivery Time","Future","Diffusion Index","NSA"],
               ["ivcdsa","Inventories","Current","Decrease","SA"],
               ["ivcdna","Inventories","Current","Decrease","NSA"],
               ["ivfdsa","Inventories","Future","Decrease","SA"],
               ["ivfdna","Inventories","Future","Decrease","NSA"],
               ["ivcnsa","Inventories","Current","No Change","SA"],
               ["ivcnna","Inventories","Current","No Change","NSA"],
               ["ivfnsa","Inventories","Future","No Change","SA"],
               ["ivfnna","Inventories","Future","No Change","NSA"],
               ["ivcisa","Inventories","Current","Increase","SA"],
               ["ivcina","Inventories","Current","Increase","NSA"],
               ["ivfisa","Inventories","Future","Increase","SA"],
               ["ivfina","Inventories","Future","Increase","NSA"],
               ["ivcdisa","Inventories","Current","Diffusion Index","SA"],
               ["ivcdina","Inventories","Current","Diffusion Index","NSA"],
               ["ivfdisa","Inventories","Future","Diffusion Index","SA"],
               ["ivfdina","Inventories","Future","Diffusion Index","NSA"],
               ["uocdsa","Unfilled Orders","Current","Decrease","SA"],
               ["uocdna","Unfilled Orders","Current","Decrease","NSA"],
               ["uofdsa","Unfilled Orders","Future","Decrease","SA"],
               ["uofdna","Unfilled Orders","Future","Decrease","NSA"],
               ["uocnsa","Unfilled Orders","Current","No Change","SA"],
               ["uocnna","Unfilled Orders","Current","No Change","NSA"],
               ["uofnsa","Unfilled Orders","Future","No Change","SA"],
               ["uofnna","Unfilled Orders","Future","No Change","NSA"],
               ["uocisa","Unfilled Orders","Current","Increase","SA"],
               ["uocina","Unfilled Orders","Current","Increase","NSA"],
               ["uofisa","Unfilled Orders","Future","Increase","SA"],
               ["uofina","Unfilled Orders","Future","Increase","NSA"],
               ["uocdisa","Unfilled Orders","Current","Diffusion Index","SA"],
               ["uocdina","Unfilled Orders","Current","Diffusion Index","NSA"],
               ["uofdisa","Unfilled Orders","Future","Diffusion Index","SA"],
               ["uofdina","Unfilled Orders","Future","Diffusion Index","NSA"],
               ["ppcdsa","Prices Paid","Current","Decrease","SA"],
               ["ppcdna","Prices Paid","Current","Decrease","NSA"],
               ["ppfdsa","Prices Paid","Future","Decrease","SA"],
               ["ppfdna","Prices Paid","Future","Decrease","NSA"],
               ["ppcnsa","Prices Paid","Current","No Change","SA"],
               ["ppcnna","Prices Paid","Current","No Change","NSA"],
               ["ppfnsa","Prices Paid","Future","No Change","SA"],
               ["ppfnna","Prices Paid","Future","No Change","NSA"],
               ["ppcisa","Prices Paid","Current","Increase","SA"],
               ["ppcina","Prices Paid","Current","Increase","NSA"],
               ["ppfisa","Prices Paid","Future","Increase","SA"],
               ["ppfina","Prices Paid","Future","Increase","NSA"],
               ["ppcdisa","Prices Paid","Current","Diffusion Index","SA"],
               ["ppcdina","Prices Paid","Current","Diffusion Index","NSA"],
               ["ppfdisa","Prices Paid","Future","Diffusion Index","SA"],
               ["ppfdina","Prices Paid","Future","Diffusion Index","NSA"],
               ["prcdsa","Prices Received","Current","Decrease","SA"],
               ["prcdna","Prices Received","Current","Decrease","NSA"],
               ["prfdsa","Prices Received","Future","Decrease","SA"],
               ["prfdna","Prices Received","Future","Decrease","NSA"],
               ["prcnsa","Prices Received","Current","No Change","SA"],
               ["prcnna","Prices Received","Current","No Change","NSA"],
               ["prfnsa","Prices Received","Future","No Change","SA"],
               ["prfnna","Prices Received","Future","No Change","NSA"],
               ["prcisa","Prices Received","Current","Increase","SA"],
               ["prcina","Prices Received","Current","Increase","NSA"],
               ["prfisa","Prices Received","Future","Increase","SA"],
               ["prfina","Prices Received","Future","Increase","NSA"],
               ["prcdisa","Prices Received","Current","Diffusion Index","SA"],
               ["prcdina","Prices Received","Current","Diffusion Index","NSA"],
               ["prfdisa","Prices Received","Future","Diffusion Index","SA"],
               ["prfdina","Prices Received","Future","Diffusion Index","NSA"],
               ["necdsa","Employment","Current","Decrease","SA"],
               ["necdna","Employment","Current","Decrease","NSA"],
               ["nefdsa","Employment","Future","Decrease","SA"],
               ["nefdna","Employment","Future","Decrease","NSA"],
               ["necnsa","Employment","Current","No Change","SA"],
               ["necnna","Employment","Current","No Change","NSA"],
               ["nefnsa","Employment","Future","No Change","SA"],
               ["nefnna","Employment","Future","No Change","NSA"],
               ["necisa","Employment","Current","Increase","SA"],
               ["necina","Employment","Current","Increase","NSA"],
               ["nefisa","Employment","Future","Increase","SA"],
               ["nefina","Employment","Future","Increase","NSA"],
               ["necdisa","Employment","Current","Diffusion Index","SA"],
               ["necdina","Employment","Current","Diffusion Index","NSA"],
               ["nefdisa","Employment","Future","Diffusion Index","SA"],
               ["nefdina","Employment","Future","Diffusion Index","NSA"],
               ["awcdsa","Workhours","Current","Decrease","SA"],
               ["awcdna","Workhours","Current","Decrease","NSA"],
               ["awfdsa","Workhours","Future","Decrease","SA"],
               ["awfdna","Workhours","Future","Decrease","NSA"],
               ["awcnsa","Workhours","Current","No Change","SA"],
               ["awcnna","Workhours","Current","No Change","NSA"],
               ["awfnsa","Workhours","Future","No Change","SA"],
               ["awfnna","Workhours","Future","No Change","NSA"],
               ["awcisa","Workhours","Current","Increase","SA"],
               ["awcina","Workhours","Current","Increase","NSA"],
               ["awfisa","Workhours","Future","Increase","SA"],
               ["awfina","Workhours","Future","Increase","NSA"],
               ["awcdisa","Workhours","Current","Diffusion Index","SA"],
               ["awcdina","Workhours","Current","Diffusion Index","NSA"],
               ["awfdisa","Workhours","Future","Diffusion Index","SA"],
               ["awfdina","Workhours","Future","Diffusion Index","NSA"],
               ["cefdsa","Capital Expenditures","Future","Decrease","SA"],
               ["cefdna","Capital Expenditures","Future","Decrease","NSA"],
               ["cefnsa","Capital Expenditures","Future","No Change","SA"],
               ["cefnna","Capital Expenditures","Future","No Change","NSA"],
               ["cefisa","Capital Expenditures","Future","Increase","SA"],
               ["cefina","Capital Expenditures","Future","Increase","NSA"],
               ["cefdisa","Capital Expenditures","Future","Diffusion Index","SA"],
               ["cefdina","Capital Expenditures","Future","Diffusion Index","NSA"],
               ["tsfdsa","Technology Spending","Future","Decrease","SA"],
               ["tsfdna","Technology Spending","Future","Decrease","NSA"],
               ["tsfnsa","Technology Spending","Future","No Change","SA"],
               ["tsfnna","Technology Spending","Future","No Change","NSA"],
               ["tsfisa","Technology Spending","Future","Increase","SA"],
               ["tsfina","Technology Spending","Future","Increase","NSA"],
               ["tsfdisa","Technology Spending","Future","Diffusion Index","SA"],
               ["tsfdina","Technology Spending","Future","Diffusion Index","NSA"]]

ny_pmi_dict = pd.DataFrame(data = ny_pmi_dict,
                           columns = ["series_id","item_name","state","data_type","seasonal"])

ny_pmi_dict.head()

,series_id,item_name,state,data_type,seasonal
0,gacdsa,General Activity,Current,Decrease,SA
1,gacdna,General Activity,Current,Decrease,NSA
2,gafdsa,General Activity,Future,Decrease,SA
3,gafdna,General Activity,Future,Decrease,NSA
4,gacnsa,General Activity,Current,No Change,SA


Saving Files

In [20]:
ny_pmi.to_parquet("data\\business_surveys\\data_ny_fed_mpmi.parquet")
ny_pmi_dict.to_parquet("data\\business_surveys\\dict_ny_fed_mpmi.parquet")